<a href="https://colab.research.google.com/github/Rama-Has/Poem_Generator_Rama_Hasiba_12010022/blob/main/Arabic_Poem_Generator_NLP_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers==4.28.0
!pip install --upgrade accelerate
!pip install git+https://github.com/huggingface/accelerate
# !pip install transformers==4.28.0 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 10.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-pouogeb4
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-pouogeb4
  Resolved https://github.com/huggingface/accelerate to commit 0226f750257b3bf2cadc4f189f9eef0c764a0467
  Installing build dependencies ... done
  Getting requirements to buil

In [4]:
!pip install pyarabic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 6.4 MB/s eta 0:00:00


In [5]:
!pip install arabic_reshaper
!pip install python-bidi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


--------------------------------------------
> # Arabic Poem Generator  




In [6]:
from google.colab import drive
import numpy as np
from google.colab import files
import shutil
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead, AutoTokenizer \
,TextDataset, DataCollatorForLanguageModeling, pipeline
from sklearn.model_selection import train_test_split
from pyarabic.araby import strip_tashkeel 
from google.colab import drive
import tensorflow as tf
import pandas as pd  
import re 
from wordcloud import WordCloud, STOPWORDS
import arabic_reshaper
from bidi.algorithm import get_display
import matplotlib.pyplot as plt

# Reading Data

In [7]:
#mount google drive to the colab runtime
drive.mount('/content/drive', force_remount = True) 

Mounted at /content/drive


In [10]:
!unzip -q "/content/drive/MyDrive/Classification Data/Arabic Poem Comprehensive Dataset (APCD).zip" -d "/content/"

In [11]:
data = pd.read_csv("/content/Arabic Poem Comprehensive Dataset (APCD).csv")
data

,العصر,الشاعر,الديوان,القافية,البحر,الشطر الايسر,الشطر الايمن,البيت
0,قبل الإسلام,عمرو بنِ قُمَيئَة,الديوان الرئيسي,د,الطويل,وَأَن تَجمَعا شَملي وَتَنتَظِرا غَدا,خَليلَيَّ لا تَستَعجِلا أَن تَزَوَّدا,خَليلَيَّ لا تَستَعجِلا أَن تَزَوَّدا وَأَن...
1,قبل الإسلام,عمرو بنِ قُمَيئَة,الديوان الرئيسي,د,الطويل,وَلا سُرعَتي يَوماً بِسابِقَةِ الرَدى,فَما لَبَثٌ يَوماً بِسابِقٍ مَغنَمٍ,فَما لَبَثٌ يَوماً بِسابِقٍ مَغنَمٍ وَلا سُ...
2,قبل الإسلام,عمرو بنِ قُمَيئَة,الديوان الرئيسي,د,الطويل,وَتَستَوجِبا مَنّاً عَلَيَّ وَتُحمَدا,وَإِن تُنظِراني اليَومَ أَقضِ لُبانَةً,وَإِن تُنظِراني اليَومَ أَقضِ لُبانَةً وَتَ...
3,قبل الإسلام,عمرو بنِ قُمَيئَة,الديوان الرئيسي,د,الطويل,تُؤامِرُني سِرّاً لِأَصرِمَ مَرثَدا,لَعَمرُكَ ما نَفسٌ بِجِدٍ رَشيدَةٍ,لَعَمرُكَ ما نَفسٌ بِجِدٍ رَشيدَةٍ تُؤامِرُ...
4,قبل الإسلام,عمرو بنِ قُمَيئَة,الديوان الرئيسي,د,الطويل,وَأَفرَعَ في لَومي مِراراً وَأَصعَدا,وَإِن ظَهَرَت مِنهُ قَوارِصُ جَمَّةٌ,وَإِن ظَهَرَت مِنهُ قَوارِصُ جَمَّةٌ وَأَفر...
...,...,...,...,...,...,...,...,...
1831765,الحديث,شهاب غانم,شهاب غانم,ن,الخفيف,وأحلى قصيدة تَتَغنى,هي أغلى ما أنشأ اللَّه في الدنيا,هي أغلى ما أنشأ اللَّه في الدنيا وأحلى قصيد...
1831766,الحديث,شهاب غانم,شهاب غانم,ن,الخفيف,كحلم يغشى الجفون الوسنى,هي أغرودة الأغاريد تنساب,هي أغرودة الأغاريد تنساب كحلم يغشى الجفون ا...
1831767,الحديث,شهاب غانم,شهاب غانم,ن,الخفيف,يتداعى وجداً ويخفق حسنا,هي شلال بهجة وبهاء,هي شلال بهجة وبهاء يتداعى وجداً ويخفق حسنا
1831768,الحديث,شهاب غانم,شهاب غانم,ن,الخفيف,يدك الحدود سجناً فسجنا,هي حلم الهوى ومنطلقي الباقي,هي حلم الهوى ومنطلقي الباقي يدك الحدود سجنا...


# General Look at the data

In [12]:
print(f"There are a atotal of {data.shape[0]} poem line in the dataset, I'll use the first 50 000 sample  and take البيت column.")

There are a atotal of 1831770 poem line in the dataset, I'll use the first 50 000 sample  and take البيت column.


In [13]:
data = data[['البيت']]
data

,البيت
0,خَليلَيَّ لا تَستَعجِلا أَن تَزَوَّدا وَأَن...
1,فَما لَبَثٌ يَوماً بِسابِقٍ مَغنَمٍ وَلا سُ...
2,وَإِن تُنظِراني اليَومَ أَقضِ لُبانَةً وَتَ...
3,لَعَمرُكَ ما نَفسٌ بِجِدٍ رَشيدَةٍ تُؤامِرُ...
4,وَإِن ظَهَرَت مِنهُ قَوارِصُ جَمَّةٌ وَأَفر...
...,...
1831765,هي أغلى ما أنشأ اللَّه في الدنيا وأحلى قصيد...
1831766,هي أغرودة الأغاريد تنساب كحلم يغشى الجفون ا...
1831767,هي شلال بهجة وبهاء يتداعى وجداً ويخفق حسنا
1831768,هي حلم الهوى ومنطلقي الباقي يدك الحدود سجنا...


In [14]:
poems_lines = data['البيت'].to_list()[: 50000]
poems_lines[0: 5]

['خَليلَيَّ لا تَستَعجِلا أَن تَزَوَّدا    وَأَن تَجمَعا شَملي وَتَنتَظِرا غَدا',
 'فَما لَبَثٌ يَوماً بِسابِقٍ مَغنَمٍ    وَلا سُرعَتي يَوماً بِسابِقَةِ الرَدى',
 'وَإِن تُنظِراني اليَومَ أَقضِ لُبانَةً    وَتَستَوجِبا مَنّاً عَلَيَّ وَتُحمَدا',
 'لَعَمرُكَ ما نَفسٌ بِجِدٍ رَشيدَةٍ    تُؤامِرُني سِرّاً لِأَصرِمَ مَرثَدا',
 'وَإِن ظَهَرَت مِنهُ قَوارِصُ جَمَّةٌ    وَأَفرَعَ في لَومي مِراراً وَأَصعَدا']

## Data Cleanning:
  1. Remove strips.
  2. Remove arabic punctuation.
  3. Remove multiple spaces.

In [15]:
# remove special characters
ARABIC_PUNCTUATION = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''

def remove_arabic_punctuation(text: str):
    translator = str.maketrans('', '', ARABIC_PUNCTUATION)
    cleaned_text = text.translate(translator)
    return cleaned_text

def remove_multiple_spaces(text):
    cleaned_text = re.sub(' +', ' ', text)
    return cleaned_text

In [16]:
poems_lines_df = data[['البيت']].head(50000).copy()
poems_lines_df['البيت'] = poems_lines_df['البيت'].map(remove_arabic_punctuation)
poems_lines_df['البيت'] = poems_lines_df['البيت'].map(strip_tashkeel)
poems_lines_df['البيت'] = poems_lines_df['البيت'].map(remove_multiple_spaces)

In [17]:
!pip install pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
from PIL import ImageFont
font = ImageFont.load_default()

In [19]:
# # Read poems
# text = " ".join(poems_lines_df['البيت'].to_list())

# # Reshape the text to join the Arabic characters correctly
# reshaped_text = arabic_reshaper.reshape(text)

# # Use the bidi algorithm to handle the right-to-left direction of Arabic text
# bidi_text = get_display(reshaped_text)

# # font = ImageFont.load_default()

# # Create the WordCloud object and generate the word cloud image
# wordcloud = WordCloud(
#     width=800, 
#     height=800, 
#     background_color='white', 
#     font_path= 'arial',#'arial.ttf',
#     min_font_size=10, 
#     max_words=200
#     ).generate(bidi_text)

# # Display the word cloud image
# plt.figure(figsize=(8, 8), facecolor=None)
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.tight_layout(pad=0)
# plt.show()

# Finetune GPT-2

In [20]:
poems_lines_df.to_csv(
    r'arabic_full_text.txt', 
    header = None, 
    index = None, 
    sep=' ', 
    mode='a'
    )

In [21]:
file_path = '/content/arabic_full_text.txt'

Splitting the data into train, and test without shuffeling.

In [22]:
train, test = train_test_split(poems_lines_df.head(50000), test_size=0.15, shuffle=False)

In [23]:
train.shape

(42500, 1)

In [24]:
train.columns

Index(['البيت'], dtype='object')

In [25]:
test.shape

(7500, 1)

Write train and test data each one into distinct text file.

In [26]:
train.to_csv(
    r'train_dataset.txt', 
    header=None, 
    index=None, 
    sep=' ', 
    mode='a'
    )

test.to_csv(
    r'test_dataset.txt', 
    header=None, 
    index=None, 
    sep=' ', 
    mode='a'
    )

In [27]:
train.head(3)

,البيت
0,خليلي لا تستعجلا أن تزودا وأن تجمعا شملي وتنتظ...
1,فما لبث يوما بسابق مغنم ولا سرعتي يوما بسابقة ...
2,وإن تنظراني اليوم أقض لبانة وتستوجبا منا علي و...


Load the tokenizer that is created specialy to the "aragpt-2base model".

In [28]:
#Load the tokenizer 
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/aragpt2-base")

train_path = '/content/train_dataset.txt'
test_path = '/content/test_dataset.txt'

In [29]:
def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )

    return train_dataset, test_dataset, data_collator

In [30]:
def remove_double_qoutes(file_path: str) -> None: 
  
    # Read the file
    with open(file_path, 'r') as file:
        text = file.read()

    # Remove double quotes
    text_without_quotes = text.replace('"', '')

    # Write the modified text back to the file
    with open(file_path, 'w') as file:
        file.write(text_without_quotes)

remove_double_qoutes(train_path)
remove_double_qoutes(test_path)

In [31]:
train_dataset, test_dataset, data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [32]:
model = AutoModelWithLMHead.from_pretrained("aubmindlab/aragpt2-base")

training_args = TrainingArguments(
    output_dir = "./gpt2-araPoems", #The output directory
    overwrite_output_dir = True, #overwrite the content of the output directory
    num_train_epochs = 15, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [33]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,7.488000
1000,6.058400
1500,5.741200
2000,5.571000


TrainOutput(global_step=2370, training_loss=6.102360026041667, metrics={'train_runtime': 3008.4565, 'train_samples_per_second': 25.114, 'train_steps_per_second': 0.788, 'total_flos': 4935479869440000.0, 'train_loss': 6.102360026041667, 'epoch': 15.0})

In [34]:
trainer.save_model('./aragpt_final_one') 

# Compress the folder into a zip file
shutil.make_archive("/content/aragpt_final_one", "zip", "/content/aragpt_final_one")


'/content/aragpt_final_one.zip'

In [35]:
generator_ = pipeline('text-generation', model='./aragpt_final_one', tokenizer='aubmindlab/aragpt2-base')

In [36]:
max_words = 50  # Set the desired maximum number of words

result = generator_('الليل ', max_length = 60)[0]['generated_text']

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [37]:
print(result)

الليل مذموم
ولقد علمتم أني إذا ما شربت من الخمر ألذ وألذ وأغلى
فإذا أنا لم أشربها فإنني سأشربها وإن كنت لا أشربها
ألم تر أن الدهر يخامره في كل يوم وليلة وفي كل يوم ويوم



## Evaluation

In [38]:
trainer.evaluate(
    test_dataset,
    metric_key_prefix = "eval_perplexity"
)

{'eval_perplexity_loss': 5.859355926513672,
 'eval_perplexity_runtime': 9.8595,
 'eval_perplexity_samples_per_second': 83.777,
 'eval_perplexity_steps_per_second': 1.319,
 'epoch': 15.0}

In [39]:
trainer.evaluate(
    test_dataset,
    metric_key_prefix = "eval_bleu"
)

{'eval_bleu_loss': 5.859355926513672,
 'eval_bleu_runtime': 10.0341,
 'eval_bleu_samples_per_second': 82.319,
 'eval_bleu_steps_per_second': 1.296,
 'epoch': 15.0}

## Evaluation

In [43]:
import pandas as pd

# Read the text file
with open('test_dataset.txt', 'r') as file:
    lines = file.readlines()

# Create a DataFrame with each row as a sample
df = pd.DataFrame({'text': lines})


df['text'] = df['text'].astype(str) 

# Remove empty rows  
df = df[df['text'] != '']


In [44]:
import tensorflow_hub as hub
import tensorflow as tf

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
embeddings = embed(
    df['text'].to_list()
)
 

In [45]:
df['embeddings'] = [embedded_sentence for embedded_sentence in embeddings]

In [50]:
def generate_embedding_for_prediction(seed_line): 
  
    predicted_line = generator_(seed_line, max_length = 20)[0]['generated_text'] 
    predicted_embedding = embed( [predicted_line] ) 
    
    return [predicted_embedding]

In [55]:
df['predicted_embedding'] = [i-i for i in range(df.shape[0])]

In [57]:
for i in range(150):
    df['predicted_embedding'].iloc[i + 1] = generate_embedding_for_prediction(df['text'].iloc[i])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for o

In [58]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
def calculate_similarity(embed1, embed2):
    # Define two vectors
    vector1 = np.array(embed1)
    vector2 = np.array(embed2)

    # Reshape the vectors to have a 2D shape
    vector1 = vector1.reshape(1, -1)
    vector2 = vector2.reshape(1, -1)

    # Calculate the cosine similarity
    similarity = cosine_similarity(vector1, vector2)

    return similarity

In [59]:
df

,text,embeddings,predicted_embedding
0,إن أنت لم تلق لي المودة في صدر حبيبي وأنت مقتدر\n,"(tf.Tensor(-0.063116215, shape=(), dtype=float...",0
1,لا قلت شعرا ولا سمعت غنا ولا جرى في مفاصلي الس...,"(tf.Tensor(0.0027435862, shape=(), dtype=float...","[((tf.Tensor(-0.06487517, shape=(), dtype=floa..."
2,ولا أزال القرآن أدرسه أروح في درسه وأبتكر\n,"(tf.Tensor(-0.024885083, shape=(), dtype=float...","[((tf.Tensor(-0.0056983815, shape=(), dtype=fl..."
3,وألزم الصوم والصلاة ولا أزال دهري بالخير آتمر\n,"(tf.Tensor(-0.070586205, shape=(), dtype=float...","[((tf.Tensor(6.4538e-05, shape=(), dtype=float..."
4,فما مضت بعد ذاك ثالثة حتى أتاني الحبيب يعتذر\n,"(tf.Tensor(-0.09535113, shape=(), dtype=float3...","[((tf.Tensor(-0.09918193, shape=(), dtype=floa..."
...,...,...,...
7495,تعللني أضاليل الأماني بعيش بعد قيصر لا يطيب\n,"(tf.Tensor(-0.022158746, shape=(), dtype=float...",0
7496,تولى العيش إذ ولى التصابي ومات الحب إذ مات الح...,"(tf.Tensor(-0.028173875, shape=(), dtype=float...",0
7497,نصيبي كان من دنياي ولى فلا الدنيا تحس ولا النص...,"(tf.Tensor(-0.0049606655, shape=(), dtype=floa...",0
7498,ضجيع مسندين بكفر توثى خفوتا مثل ما خفت الشروب\n,"(tf.Tensor(-0.05765654, shape=(), dtype=float3...",0


In [60]:
df['similarity'] = [i-i for i in range(df.shape[0])]

In [61]:
df = df[df['predicted_embedding'].notna()]

In [64]:
for i in range(1, 150): 
    predicted_embedding = df['predicted_embedding'].iloc[i][0]
    ground_truth_embedding = df['predicted_embedding'].iloc[i + 1][0]

    df['similarity'].iloc[i] = calculate_similarity(predicted_embedding, ground_truth_embedding)

<ipython-input-64-8e6fa34c5492>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['similarity'].iloc[i] = calculate_similarity(predicted_embedding, ground_truth_embedding)


In [66]:
df.head(150)

,text,embeddings,predicted_embedding,similarity
0,إن أنت لم تلق لي المودة في صدر حبيبي وأنت مقتدر\n,"(tf.Tensor(-0.063116215, shape=(), dtype=float...",0,0.000000
1,لا قلت شعرا ولا سمعت غنا ولا جرى في مفاصلي الس...,"(tf.Tensor(0.0027435862, shape=(), dtype=float...","[((tf.Tensor(-0.06487517, shape=(), dtype=floa...",0.764992
2,ولا أزال القرآن أدرسه أروح في درسه وأبتكر\n,"(tf.Tensor(-0.024885083, shape=(), dtype=float...","[((tf.Tensor(-0.0056983815, shape=(), dtype=fl...",0.920483
3,وألزم الصوم والصلاة ولا أزال دهري بالخير آتمر\n,"(tf.Tensor(-0.070586205, shape=(), dtype=float...","[((tf.Tensor(6.4538e-05, shape=(), dtype=float...",0.367782
4,فما مضت بعد ذاك ثالثة حتى أتاني الحبيب يعتذر\n,"(tf.Tensor(-0.09535113, shape=(), dtype=float3...","[((tf.Tensor(-0.09918193, shape=(), dtype=floa...",0.543063
...,...,...,...,...
145,فقد جنيت الهموم منه وقد خليت قلبي يعوم في الفكر\n,"(tf.Tensor(-0.03576632, shape=(), dtype=float3...","[((tf.Tensor(0.0036948186, shape=(), dtype=flo...",0.855345
146,لا أسعد القلب في هواه ولا يطمع في عزتي ولا خوري\n,"(tf.Tensor(0.0029054608, shape=(), dtype=float...","[((tf.Tensor(-0.042879425, shape=(), dtype=flo...",0.831667
147,عف ضميري وطيب خبري ولذتي في الحديث والنظر\n,"(tf.Tensor(-0.032108586, shape=(), dtype=float...","[((tf.Tensor(0.0072834273, shape=(), dtype=flo...",0.891139
148,هارون يا خير الخلائف كلهم ممن مضى فيهم وهذا ال...,"(tf.Tensor(-0.09284633, shape=(), dtype=float3...","[((tf.Tensor(-0.045009732, shape=(), dtype=flo...",0.291991
